[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/NktBkzn/HSE_DL_2021/blob/master/12_week/RNNs.ipynb)

- Ноутбук заимствован с курса [dlcourse.ai](https://dlcourse.ai)
- Решение ноутбука взято [отсюда](https://github.com/omega1996/dlcourse/blob/master/assignments/assignment6/RNNs.ipynb)

# Задание 6: Рекуррентные нейронные сети (RNNs)

Это задание адаптиповано из Deep NLP Course at ABBYY (https://github.com/DanAnastasyev/DeepNLP-Course) с разрешения автора - Даниила Анастасьева. Спасибо ему огромное!

In [ ]:
!pip install gensim==4.1.2
!pip install nltk==3.6.3
!pip install scikit-learn==1.0.1

In [ ]:
import nltk
nltk.__version__

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
else:
    from torch import FloatTensor, LongTensor

np.random.seed(42)

# Рекуррентные нейронные сети (RNNs)

## POS Tagging

Мы рассмотрим применение рекуррентных сетей к задаче sequence labeling (последняя картинка).

![RNN types](http://karpathy.github.io/assets/rnn/diags.jpeg)

*From [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*

Самые популярные примеры для такой постановки задачи - Part-of-Speech Tagging и Named Entity Recognition.

Мы порешаем сейчас POS Tagging для английского.

Будем работать с таким набором тегов:
- ADJ - adjective (new, good, high, ...)
- ADP - adposition (on, of, at, ...)
- ADV - adverb (really, already, still, ...)
- CONJ - conjunction (and, or, but, ...)
- DET - determiner, article (the, a, some, ...)
- NOUN - noun (year, home, costs, ...)
- NUM - numeral (twenty-four, fourth, 1991, ...)
- PRT - particle (at, on, out, ...)
- PRON - pronoun (he, their, her, ...)
- VERB - verb (is, say, told, ...)
- . - punctuation marks (. , ;)
- X - other (ersatz, esprit, dunno, ...)

Скачаем данные:

In [ ]:
import nltk
from sklearn.model_selection import train_test_split

nltk.download('brown')
nltk.download('universal_tagset')

data = nltk.corpus.brown.tagged_sents(tagset='universal')

In [ ]:
len(data), len(data[0]), len(data[1])

# list(list(tuple))

# (word, tag)

Пример размеченного предложения:

In [ ]:
for word, tag in data[0]:
    print('{:15}\t{}'.format(word, tag))

Построим разбиение на train/val/test - наконец-то, всё как у нормальных людей.

На train будем учиться, по val - подбирать параметры и делать всякие early stopping, а на test - принимать модель по ее финальному качеству.

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.15, random_state=42)

print('Words count in train set:', sum(len(sent) for sent in train_data))
print('Words count in val set:', sum(len(sent) for sent in val_data))
print('Words count in test set:', sum(len(sent) for sent in test_data))

Построим маппинги из слов в индекс и из тега в индекс:


In [ ]:
words = {word for sample in train_data for word, tag in sample}
word2ind = {word: ind + 1 for ind, word in enumerate(words)}  # ind + 1 to leave 0 idx blank for <pad>
word2ind['<pad>'] = 0

tags = {tag for sample in train_data for word, tag in sample}
tag2ind = {tag: ind + 1 for ind, tag in enumerate(tags)}
tag2ind['<pad>'] = 0

print('Unique words in train = {}. Tags = {}'.format(len(word2ind), tags))

In [ ]:
word2ind

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

tag_distribution = Counter(tag for sample in train_data for _, tag in sample)
tag_distribution = [tag_distribution[tag] for tag in tags]

plt.figure(figsize=(10, 5))

bar_width = 0.35
plt.bar(np.arange(len(tags)), tag_distribution, bar_width, align='center', alpha=0.5)
plt.xticks(np.arange(len(tags)), tags)
    
plt.show()

## Бейзлайн

Какой самый простой теггер можно придумать? Давайте просто запоминать, какие теги самые вероятные для слова (или для последовательности):

![tag-context](https://www.nltk.org/images/tag-context.png)  
*From [Categorizing and Tagging Words, nltk](https://www.nltk.org/book/ch05.html)*

На картинке показано, что для предсказания $t_n$ используются два предыдущих предсказанных тега + текущее слово. По корпусу считаются вероятность для $P(t_n| w_n, t_{n-1}, t_{n-2})$, выбирается тег с максимальной вероятностью.

Более аккуратно такая идея реализована в Hidden Markov Models: по тренировочному корпусу вычисляются вероятности $P(w_n| t_n), P(t_n|t_{n-1}, t_{n-2})$ и максимизируется их произведение.

Простейший вариант - униграммная модель, учитывающая только слово:

На вход UnigramTagger принимает данные в таком формате:\
*train (list(list(tuple(str, str)))) – The corpus of training data, a list of tagged sentences*

In [ ]:
test_data[:2]

In [ ]:
import nltk

default_tagger = nltk.DefaultTagger('NN')  # tags every word witn a noun

# The UnigramTagger finds the most likely tag for each word in a training corpus, 
# and then uses that information to assign tags to new tokens.
unigram_tagger = nltk.UnigramTagger(train_data, backoff=default_tagger)
print('Accuracy of unigram tagger = {:.2%}'.format(unigram_tagger.evaluate(test_data)))

Хорошая [заметка](https://stackoverflow.com/questions/46713629/evaluating-pos-tagger-in-nltk) на SO про синтаксис Taggers из nltk

Выжимка:
```python
tagged_sentences = brown.tagged_sents(categories="news", tagset="universal")

# let's keep 20% of the data for testing, and 80 for training
i = int(len(tagged_sentences)*0.2)
train_sentences = tagged_sentences[i:]
test_sentences = tagged_sentences[:i]

# train
unigram_tagger = UnigramTagger(train_sentences)
# get ACCURACY; default evaluation metric for nltk taggers is accuracy
accuracy = unigram_tagger.evaluate(test_sentences)

tagged_test_sentences = unigram_tagger.tag_sents([[token for token,tag in sent] for sent in test_sentences])  # words(tokens) only
pred = [str(tag) for sentence in tagged_test_sentences for token,tag in sentence]  # predicted tags
gold = [str(tag) for sentence in test_sentences for token,tag in sentence]  # true tags

from sklearn import metrics
print(metrics.classification_report(gold, pred))
```

Добавим вероятности переходов:

In [ ]:
# A tagger that chooses a token’s tag based its word string and on the preceding 
# words’ tag. In particular, a tuple consisting of the previous tag and 
# the word is looked up in a table, and the corresponding tag is returned.
bigram_tagger = nltk.BigramTagger(train_data, backoff=unigram_tagger)
print('Accuracy of bigram tagger = {:.2%}'.format(bigram_tagger.evaluate(test_data)))

Обратите внимание, что `backoff` важен:

In [ ]:
trigram_tagger = nltk.TrigramTagger(train_data)
print('Accuracy of trigram tagger = {:.2%}'.format(trigram_tagger.evaluate(test_data)))

In [ ]:
trigram_tagger = nltk.TrigramTagger(train_data, backoff=unigram_tagger)
print('Accuracy of trigram tagger = {:.2%}'.format(trigram_tagger.evaluate(test_data)))

## Увеличиваем контекст с рекуррентными сетями

Униграмная модель работает на удивление хорошо, но мы же собрались учить сеточки.

Омонимия - основная причина, почему униграмная модель плоха:  
*“he cashed a check at the **bank**”*  
vs  
*“he sat on the **bank** of the river”*

Поэтому нам очень полезно учитывать контекст при предсказании тега.

Воспользуемся LSTM - он умеет работать с контекстом очень даже хорошо:

![](https://image.ibb.co/kgmoff/Baseline-Tagger.png)

Синим показано выделение фичей из слова, LSTM оранжевенький - он строит эмбеддинги слов с учетом контекста, а дальше зелененькая логистическая регрессия делает предсказания тегов.

In [ ]:
def convert_data(data, word2ind, tag2ind):
    X = [[word2ind.get(word, 0) for word, _ in sample] for sample in data]
    y = [[tag2ind[tag] for _, tag in sample] for sample in data]
    
    return X, y

X_train, y_train = convert_data(train_data, word2ind, tag2ind)
X_val, y_val = convert_data(val_data, word2ind, tag2ind)
X_test, y_test = convert_data(test_data, word2ind, tag2ind)

In [ ]:
X_train[0]

In [ ]:
len(X_train), len(train_data)

# len(data[0]), len(data[1])

In [ ]:
def iterate_batches(data, batch_size):
    X, y = data
    n_samples = len(X)
    # print('n_samples', n_samples)
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        # Создаем матрицы, заполненные нулями, которые ниже будут заполнены значениями
        batch_indices = indices[start:end]
        # print('batch_indices', batch_indices)
        max_sent_len = max(len(X[ind]) for ind in batch_indices)
        # print('max_sent_len', max_sent_len)
        X_batch = np.zeros((max_sent_len, len(batch_indices)))
        y_batch = np.zeros((max_sent_len, len(batch_indices)))
        
        # заполнение созданных матриц значениями (с учетом того, что все предложения разной длины)
        for batch_ind, sample_ind in enumerate(batch_indices):
            X_batch[:len(X[sample_ind]), batch_ind] = X[sample_ind]
            y_batch[:len(y[sample_ind]), batch_ind] = y[sample_ind]
            
        yield X_batch, y_batch

In [ ]:
X_batch, y_batch = next(iterate_batches((X_train, y_train), 4))

X_batch.shape, y_batch.shape

In [ ]:
print(X_batch[:2])

print(y_batch[:2])

**Задание** Реализуйте `LSTMTagger`:

* [nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
    - Input: (∗)
    - Output: (∗, H)
* [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
    - Input: (N, L, H_in) или (L, N, H_in) в зависимости от значения `batch_first`
    - Output: (N, L, D * H_out), (h_n, c_n)\
    D = 2 if bidirectional=True otherwise 1

$L\times bs \xrightarrow{\text{nn.Embedding}} L \times bs \times H_{in} \xrightarrow{\text{nn.LSTM}} L \times bs \times H^*_{out}\xrightarrow{\text{nn.Linear}} L \times bs \times \text{tagset_size}$ 

\*$H_{out}$ = lstm_hidden_dim

- L - sequence length
- bs - batch size
- $H_{in}$ - number of input features representing an object
- $H_{out}$ - number of output features representing an object

In [ ]:
class LSTMTagger(nn.Module):
    def __init__(self, vocab_size, tagset_size, word_emb_dim=100, lstm_hidden_dim=128, lstm_layers_count=1):
        super().__init__()
        
        self._emb = nn.Embedding(vocab_size, word_emb_dim)
        self._lstm = nn.LSTM(word_emb_dim, lstm_hidden_dim, num_layers=lstm_layers_count)
        self._out_layer = nn.Linear(lstm_hidden_dim, tagset_size)

    def forward(self, inputs):
        emb = self._emb(inputs)
        # print('emb shape:', emb.shape)
        output, _ = self._lstm(emb)
        out = self._out_layer(output)
        return out

**Задание** Научитесь считать accuracy и loss (а заодно проверьте, что модель работает)

In [ ]:
model = LSTMTagger(
    vocab_size=len(word2ind),
    tagset_size=len(tag2ind)
)

X_batch, y_batch = torch.LongTensor(X_batch), torch.LongTensor(y_batch)

logits = model(X_batch)
preds = torch.argmax(logits, dim=-1)

# accuracy
mask = (y_batch != 0).float() # помним, что тэг 0 соотвествует слову <pad>, не учитываем <pad> в рассчете точности
correct_count = ((preds == y_batch).float() * mask).sum().item()
total_count = mask.sum().item()

print(f'Accuracy: {correct_count / total_count:.4f}')

In [ ]:
print(X_batch.shape, logits.shape, preds.shape)
# 32x4 ->(embed) 32x4x100 ->(lstm) 32x4x128 ->(linear) 32x4x13
preds[:2]

In [ ]:
print(y_batch.shape)
y_batch

[nn.CrossEnropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html?highlight=ignore_index)

$LogLoss = -\sum_{i=1}^{C} t_i log(p_i)$

$\ell(x, y)=L=\left\{l_{1}, \ldots, l_{N}\right\}^{\top}, \quad l_{n}=-w_{y_{n}} \log \dfrac{\exp \left(x_{n, y_{n}}\right)}{\sum_{c=1}^{C} \exp \left(x_{n, c}\right)} \cdot 1\left\{y_{n} \neq\right. \text { ignore_index }\}$

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0) # при расчете Loss не учитывается 0-tag
logits = model(X_batch)
loss = 0
for ind, row in enumerate(logits):
    loss += criterion(row, y_batch[ind])  # Input: (N, C) & (N) where C - classes #
print(loss)

**Задание** Вставьте эти вычисление в функцию:

In [ ]:
import math
from tqdm import tqdm


def do_epoch(model, criterion, data, batch_size, optimizer=None, name=None):
    epoch_loss = 0
    correct_count = 0
    sum_count = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = math.ceil(len(data[0]) / batch_size)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, (X_batch, y_batch) in enumerate(iterate_batches(data, batch_size)):
                X_batch, y_batch = LongTensor(X_batch), LongTensor(y_batch)
                logits = model(X_batch)

                loss = 0
                for ind, row in enumerate(logits):
                    loss += criterion(row, y_batch[ind])

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                preds = torch.argmax(logits, dim=-1)
                mask = (y_batch != 0).float()
                
                cur_correct_count, cur_sum_count = ((preds == y_batch).float() * mask).sum().item(), mask.sum().item()

                correct_count += cur_correct_count
                sum_count += cur_sum_count

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
                    name, loss.item(), cur_correct_count / cur_sum_count)
                )
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
                name, epoch_loss / batches_count, correct_count / sum_count)
            )

    return epoch_loss / batches_count, correct_count / sum_count


def fit(model, criterion, optimizer, train_data, epochs_count=1, batch_size=32,
        val_data=None, val_batch_size=None):
        
    if not val_data is None and val_batch_size is None:
        val_batch_size = batch_size
        
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss, train_acc = do_epoch(model, criterion, train_data, batch_size, optimizer, name_prefix + 'Train:')
        
        if not val_data is None:
            val_loss, val_acc = do_epoch(model, criterion, val_data, val_batch_size, None, name_prefix + '  Val:')

In [ ]:
model = LSTMTagger(
    vocab_size=len(word2ind),
    tagset_size=len(tag2ind)
).cuda()

criterion = nn.CrossEntropyLoss(ignore_index=0).cuda()
optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_data=(X_train, y_train), epochs_count=5,
    batch_size=64, val_data=(X_val, y_val), val_batch_size=512)

### Masking

**Задание** Проверьте себя - не считаете ли вы потери и accuracy на паддингах - очень легко получить высокое качество за счет этого.

У функции потерь есть параметр `ignore_index`, для таких целей. Для accuracy нужно использовать маскинг - умножение на маску из нулей и единиц, где нули на позициях паддингов (а потом усреднение по ненулевым позициям в маске).

**Задание** Посчитайте качество модели на тесте. Ожидается результат лучше бейзлайна!

In [ ]:
def compute_accuracy(model, data, batch_size=64):
    model.eval()
    val_accuracy = 0
    correct = 0
    total = 0
    for i, (X_batch, y_batch) in enumerate(iterate_batches(data, batch_size)):
        X_batch, y_batch = LongTensor(X_batch), LongTensor(y_batch)
        logits = model(X_batch)
        
        pred = torch.argmax(logits, dim=-1)
        mask = (y_batch != 0).float()
        
        correct += ((pred == y_batch).float() * mask).sum().item()
        
        total += mask.sum().item()        
        
    val_accuracy = float(correct)/total
        
    return val_accuracy

test_ac =  compute_accuracy(model, (X_test, y_test))
print(f'Test accuracy is {test_ac:.2%}')

### Bidirectional LSTM

Благодаря BiLSTM можно использовать сразу оба контеста при предсказании тега слова. Т.е. для каждого токена $w_i$ forward LSTM будет выдавать представление $\mathbf{f_i} \sim (w_1, \ldots, w_i)$ - построенное по всему левому контексту - и $\mathbf{b_i} \sim (w_n, \ldots, w_i)$ - представление правого контекста. Их конкатенация автоматически захватит весь доступный контекст слова: $\mathbf{h_i} = [\mathbf{f_i}, \mathbf{b_i}] \sim (w_1, \ldots, w_n)$.

![BiLSTM](https://www.researchgate.net/profile/Wang_Ling/publication/280912217/figure/fig2/AS:391505383575555@1470353565299/Illustration-of-our-neural-network-for-POS-tagging.png)  
*From [Finding Function in Form: Compositional Character Models for Open Vocabulary Word Representation](https://arxiv.org/abs/1508.02096)*

**Задание** Добавьте Bidirectional LSTM.

Вспомним, что Unidirectional LSTM выглядела так:
```python
class LSTMTagger(nn.Module):
    def __init__(self, vocab_size, tagset_size, word_emb_dim=100, lstm_hidden_dim=128, lstm_layers_count=1):
        super().__init__()
        
        self._emb = nn.Embedding(vocab_size, word_emb_dim)
        self._lstm = nn.LSTM(word_emb_dim, lstm_hidden_dim, num_layers=lstm_layers_count)
        self._out_layer = nn.Linear(lstm_hidden_dim, tagset_size)

    def forward(self, inputs):
        emb = self._emb(inputs)
        # print('emb shape:', emb.shape)
        output, _ = self._lstm(emb)
        out = self._out_layer(output)
        return out
```

$L\times bs \xrightarrow{\text{nn.Embedding}} L \times bs \times H_{in} \xrightarrow{\text{nn.LSTM}} L \times bs \times D\cdot H_{out}\xrightarrow{\text{nn.Linear}} L \times bs \times \text{tagset_size}$ 

In [ ]:
class BidirectionalLSTMTagger(nn.Module):
    def __init__(self, vocab_size, tagset_size, word_emb_dim=100, lstm_hidden_dim=128, lstm_layers_count=1):
        super().__init__()
        
        self._emb = nn.Embedding(vocab_size, word_emb_dim)
        self._lstm = nn.LSTM(word_emb_dim, 
                             lstm_hidden_dim, 
                             num_layers=lstm_layers_count,
                             bidirectional = True)
        self._out_layer = nn.Linear(2*lstm_hidden_dim, tagset_size)

    def forward(self, inputs):
        emb = self._emb(inputs)
        output, _ = self._lstm(emb)
        out = self._out_layer(output)
        return out

In [ ]:
model = BidirectionalLSTMTagger(
    lstm_layers_count=2,
    vocab_size=len(word2ind),
    tagset_size=len(tag2ind)
).cuda()

criterion = nn.CrossEntropyLoss(ignore_index = 0).cuda()
optimizer = optim.Adam(model.parameters(), lr = 5e-3, weight_decay = 5e-4)

fit(model, criterion, optimizer, train_data=(X_train, y_train), 
    epochs_count=6, batch_size=64, val_data=(X_val, y_val), val_batch_size=512)

### Предобученные эмбеддинги

Мы знаем, какая клёвая вещь - предобученные эмбеддинги. При текущем размере обучающей выборки еще можно было учить их и с нуля - с меньшей было бы совсем плохо.

Поэтому стандартный пайплайн - скачать эмбеддинги, засунуть их в сеточку. Запустим его:

In [ ]:
import gensim.downloader as api

w2v_model = api.load('glove-wiki-gigaword-100')

In [ ]:
w2v_model.vectors.shape
w2v_model.get_vector('is').shape

Построим подматрицу для слов из нашей тренировочной выборки:

In [ ]:
w2v_model

In [ ]:
known_count = 0
embeddings = np.zeros((len(word2ind), w2v_model.vectors.shape[1]))
for word, ind in word2ind.items():
    word = word.lower()
    if word in w2v_model.key_to_index:
        embeddings[ind] = w2v_model.get_vector(word)
        known_count += 1
        
print('Know {} out of {} word embeddings'.format(known_count, len(word2ind)))

**Задание** Сделайте модель с предобученной матрицей. Используйте `nn.Embedding.from_pretrained`.

In [ ]:
embeddings.shape

In [ ]:
embeddings_t = torch.from_numpy(embeddings)
embeddings_t.requires_grad = True
embeddings_t = embeddings_t.float().cuda()

In [ ]:
class LSTMTaggerWithPretrainedEmbs(nn.Module):
    def __init__(self, embeddings, tagset_size, lstm_hidden_dim=64, lstm_layers_count=1):
        super().__init__()
        
        self._emb = nn.Embedding.from_pretrained(embeddings_t) # важно: по дефолту freeze=True, embeddings не обучаются!
        self._lstm = nn.LSTM(embeddings_t.shape[1], 
                             lstm_hidden_dim, 
                             num_layers=lstm_layers_count, 
                             bidirectional = True)
        self._out_layer = nn.Linear(2*lstm_hidden_dim, tagset_size)

    def forward(self, inputs):
        emb = self._emb(inputs)
        output, _ = self._lstm(emb)
        out = self._out_layer(output)
        return out

In [ ]:
model = LSTMTaggerWithPretrainedEmbs(
    embeddings=embeddings,
    tagset_size=len(tag2ind)
).cuda()

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_data=(X_train, y_train), epochs_count=35,
    batch_size=64, val_data=(X_val, y_val), val_batch_size=512)

**Задание** Оцените качество модели на тестовой выборке. Обратите внимание, вовсе не обязательно ограничиваться векторами из урезанной матрицы - вполне могут найтись слова в тесте, которых не было в трейне и для которых есть эмбеддинги.

Добейтесь качества лучше прошлых моделей.

In [ ]:
test_ac =  compute_accuracy(model, (X_test, y_test))
print(f'Test accuracy is {test_ac:.2%}')

### Сравнение результатов работы разных подходов
| №      | Название        | Точность
| :---:  |:-------------:  | :------------------ | 
| 1      | Unigram Tagger  | 92.62%              | 
| 2      | Bigram Tagger   | 93.42%              | 
| 3      | Trigram Tagger  | 93.28%              | 
| 4      | LSTM            | 94.02%              | 
| 5      | BiLSTM          | 95.42%              | 
| 6      | BiLSTM Pretrained embeddings| 96.82%  |